Image is a matrix of numbers
A grayscale image is a two-dimensional matrix with values ranging from 0 (black) to 255(white)
Each pixel was a feature, read the grayscale image using OpenCV or Python-PIL, convert to a numpy array and ravel(flatten) the matrix
RGB images have three matrics instead of one
  

In [ ]:
#generates a random matrix using numpy 
import numpy as np 
import matplotlib.pyplot as plt

#gen random numpy arr with values from 0 to 255, size 256x256
random_image = np.random.randint(0, 256, (256,256))
#initalize plot 
plt.figure(figsize=(7,7))
#show grayscale image
plt.imshow(random_image, cmap='gray', vmin =0, vmax=255)
plt.show()

Building a model to detect if an X-ray image of a lung has pneumothorax or not
with dataset: https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation
Dataset consists of 10675 unique images and 2379 have pneunomthorax
---
- Binary classification with skewed data >> choose evaluation metric AUC and stratified k-fold cross-validation 
-  the images are of size 1024x1024 >> take a long time to train a model >> resize the images to 256x256 

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from tqdm import tqdm


def create_dataset(training_df, image_dir):
    """
    training_df: dataframe with ImageId, Target columns
    image_dir: location of images (folder), string
    return: X, y (training array with features and labels)
    """
    # create empty list to store image vectors
    images = []
    # create empty list to store targets
    targets = []
    # loop over the dataframe
    for index, row in tqdm(
        training_df.iterrows(),
        total=len(training_df),
        desc="processing images"
    ):
        # get image id
        image_id = row["ImageId"]
        # create image path
        image_path = os.path.join(image_dir, image_id)
        # open image using PIL
        image = Image.open(image_path + ".png")
        # resize image to 256x256. we use bilinear resampling
        image = image.resize((256, 256), resample=Image.BILINEAR)
        # convert image to array
        image = np.array(image)
        # ravel
        image = image.ravel()
        # append images and targets lists
        images.append(image)
        targets.append(int(row["target"]))
    # convert list of list of images to numpy array
    images = np.array(images)
    # print size of this array
    print(images.shape)
    return images, targets
if __name__ == "__main__":
    csv_path = "C:\Users\thomtt10\Documents\PRACTICE_CODE\Approach_book\Img_clasify_segment\input\siim_train.csv"
    image_path = "C:\Users\thomtt10\Documents\PRACTICE_CODE\Approach_book\Img_clasify_segment\input\siim_train_png/"
    # read CSV with imageid and target columns
    df = pd.read_csv(csv_path)
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # the next step is to randomize the rows of the data
    df = df.sample(frac=1).reset_index(drop=True)
    # fetch labels
    y = df.target.values
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)
    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f
    # we go over the folds created
    for fold_ in range(5):
        # temporary dataframes for train and test
        train_df = df[df.kfold != fold_].reset_index(drop=True)
        test_df = df[df.kfold == fold_].reset_index(drop=True)
        # create train dataset
        # you can move this outside to save some computation time
        xtrain, ytrain = create_dataset(train_df, image_path)
        # create test dataset
        # you can move this outside to save some computation time
        xtest, ytest = create_dataset(test_df, image_path)
        # fit random forest without any modification of params
        clf = ensemble.RandomForestClassifier(n_jobs=-1)
        clf.fit(xtrain, ytrain)
        # predict probability of class 1
        preds = clf.predict_proba(xtest)[:, 1]
        # print results
        print(f"FOLD: {fold_}")
        print(f"AUC = {metrics.roc_auc_score(ytest, preds)}")
       

Filters: two-dimensional matrices which are initialized by a given function 
Activation function: ReLU(Rectified Linear Units) and proper initialization is required to avoid the problem of vanishing gradients (when gradients approach zero and weights of network do not change)

Filter is convolved with the image. Convolution is  a summation of elementwise multiplication (crosscorrelation) between the filter and the pixels it is currently overlapping in a given
image
